In [ ]:
import pickle

In [ ]:
tokenized_stores = {'en_train': [], 'en_dev': [], 'en_test': [], 'de_train': [], 'de_dev': [], 'de_test': []}

In [ ]:

for key in tokenized_stores:
    file_name = "/content/sample_data/" + str(key)[3:] + "." + str(key)[0:2]
    load = open(file_name)
    sentences = load.read().split('\n')
    
    for sentence in sentences:
        token_store = sentence.split(' ')
        tokenized_stores[key].append(token_store)

In [ ]:
print(tokenized_stores['de_train'][2])

['Ein', 'kleines', 'Mädchen', 'klettert', 'in', 'ein', 'Spielhaus', 'aus', 'Holz.']


In [ ]:
train_size = len(tokenized_stores['en_train'])
dev_size = len(tokenized_stores['en_dev'])
test_size = len(tokenized_stores['en_test'])

In [ ]:


en_words = {}
de_words = {}

for key in tokenized_stores:
    if str(key)[0] == 'e':
        # creating en_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in en_words:
                    en_words[word] += 1
                else:
                    en_words[word] = 1
    else:
        # creating de_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in de_words:
                    de_words[word] += 1
                else:
                    de_words[word] = 1
                    
en_vocab = len(en_words)
de_vocab = len(de_words)
print("Number of Unique Words:")
print("> English:", str(en_vocab))
print("> German:", str(de_vocab))

Number of Unique Words:
> English: 15955
> German: 25871


In [ ]:

t = {}

uniform = 1 / (en_vocab * de_vocab)

In [ ]:
n_iters = 0
max_iters = 25

fine_tune = 1
has_converged = False

while n_iters < max_iters and has_converged == False:
    has_converged = True
    max_change = -1

    n_iters += 1
    count = {}
    total = {}
    for index in range(train_size):
        s_total = {}
        for en_word in tokenized_stores['en_train'][index]:
            s_total[en_word] = 0
            for de_word in tokenized_stores['de_train'][index]:
                if (en_word, de_word) not in t:
                    t[(en_word, de_word)] = uniform
                s_total[en_word] += t[(en_word, de_word)]

        for en_word in tokenized_stores['en_train'][index]:
            for de_word in tokenized_stores['de_train'][index]:
                if (en_word, de_word) not in count:
                    count[(en_word, de_word)] = 0
                count[(en_word, de_word)] += (t[(en_word, de_word)] / s_total[en_word])

                if de_word not in total:
                    total[de_word] = 0
                total[de_word] += (t[(en_word, de_word)] / s_total[en_word])



    if fine_tune == 0:
      updated = {}
   
      for index in range(train_size):
          for de_word in tokenized_stores['hi_train'][index]:
              for en_word in tokenized_stores['en_train'][index]:
                  if (en_word, de_word) in updated:
                      continue
                  updated[(en_word, de_word)] = 1
                  if abs(t[(en_word, de_word)] - count[(en_word, de_word)] / total[de_word]) > 0.01:
                      has_converged = False
                      max_change = max(max_change, abs(t[(en_word, de_word)] - count[(en_word, de_word)] / total[de_word]))
                  t[(en_word, de_word)] = count[(en_word, de_word)] / total[de_word]

    elif fine_tune == 1:
 
      max_words = 1000
      n_de_words = 0
      updates = 0

      for de_word_tuples in sorted(de_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
          de_word = de_word_tuples[0]
          n_de_words += 1
          if n_de_words > max_words:
              break
          n_en_words = 0
          for en_word_tuples in sorted(en_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
              en_word = en_word_tuples[0]
              n_en_words += 1
              if n_en_words > max_words:
                  break
              if (en_word, de_word) not in count or de_word not in total:
                  continue
               
              else:
                  if abs(t[(en_word, de_word)] - count[(en_word, de_word)] / total[de_word]) > 0.005:
                      has_converged = False
                      max_change = max(max_change, abs(t[(en_word, de_word)] - count[(en_word, de_word)] / total[de_word]))
                  t[(en_word, de_word)] = count[(en_word, de_word)] / total[de_word]

    print("Iteration " + str(n_iters) + " Completed, Maximum Change: " + str(max_change))


Iteration 1 Completed, Maximum Change: 0.1597803700363949
Iteration 2 Completed, Maximum Change: 0.40248301121761515
Iteration 3 Completed, Maximum Change: 0.20909021061251126
Iteration 4 Completed, Maximum Change: 0.12164838901266967
Iteration 5 Completed, Maximum Change: 0.07559706500395114
Iteration 6 Completed, Maximum Change: 0.05774360390926725
Iteration 7 Completed, Maximum Change: 0.04442068505958946
Iteration 8 Completed, Maximum Change: 0.03396692761756831
Iteration 9 Completed, Maximum Change: 0.026268763117816674
Iteration 10 Completed, Maximum Change: 0.02067943052064314
Iteration 11 Completed, Maximum Change: 0.01658706280131461
Iteration 12 Completed, Maximum Change: 0.01703337264823094
Iteration 13 Completed, Maximum Change: 0.01657364960472571
Iteration 14 Completed, Maximum Change: 0.013888085258384295
Iteration 15 Completed, Maximum Change: 0.01703985188298037
Iteration 16 Completed, Maximum Change: 0.016479172930858273
Iteration 17 Completed, Maximum Change: 0.01252

In [ ]:

limit = 40
for element in sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True):
  print(element)
  limit -= 1
  if limit <= 0:
    break

(('red', 'roten'), 0.8235094747427799)
(('blue', 'blauen'), 0.8150490296931527)
(('white', 'weißen'), 0.8118330005873822)
(('black', 'schwarzen'), 0.8094074052314926)
(('green', 'grünen'), 0.808763792356389)
(('yellow', 'gelben'), 0.7971283642963319)
(('black', 'schwarzer'), 0.7835584023233995)
(('white', 'weißer'), 0.763299710672295)
(('man', 'Mann'), 0.749633859772302)
(('two', 'zwei'), 0.7462514211008411)
(('young', 'junge'), 0.7454239789965208)
(('pink', 'rosa'), 0.7235349133858624)
(('Two', 'Zwei'), 0.7182701695696694)
(('behind', 'hinter'), 0.7170582276657387)
(('gray', 'grauen'), 0.7093538243552636)
(('young', 'junger'), 0.7013400229337147)
(('black', 'schwarze'), 0.6961147864434496)
(('dogs', 'Hunde'), 0.6902492812137228)
(('dog', 'Hund'), 0.6898722216364478)
(('a', 'einem'), 0.6898471649232566)
(('brown', 'brauner'), 0.6885520380249324)
(('Three', 'Drei'), 0.6850306082089465)
(('a', 'einen'), 0.6819406179153034)
(('boy', 'Junge'), 0.6809314891983571)
(('woman', 'Frau'), 0.6771

In [ ]:

file = open("translation_model.pkl","wb")
pickle.dump(t, file)
file.close()

In [ ]:

model_name = "translation_model.pkl"
pickle_in = open(model_name,"rb")
t = pickle.load(pickle_in)

In [ ]:
I = {}
for index in range(train_size):
    for en_id in range(len(tokenized_stores['en_train'][index])):
        length = len(tokenized_stores['en_train'][index])
        if length not in I:
            I[length] = {} # maps the positional difference to a tuple: (sum of t's, count)
        for hi_id in range(len(tokenized_stores['de_train'][index])):
            if (hi_id - en_id) not in I[length]:
                I[length][(hi_id - en_id)] = [t[(tokenized_stores['en_train'][index][en_id], tokenized_stores['de_train'][index][hi_id])], 1]
            else:
                I[length][(hi_id - en_id)][0] += t[(tokenized_stores['en_train'][index][en_id], tokenized_stores['de_train'][index][hi_id])]
                I[length][(hi_id - en_id)][1] += 1

In [ ]:

sentence_lengths = []
for key in I.keys():
    if key not in sentence_lengths:
        sentence_lengths.append(key)
sentence_lengths.sort()
print(sentence_lengths)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37]


In [ ]:

 
p = {}
for key in I.keys():
    p[key] = {}
    sum_val = 0
    for diff in I[key].keys():
        p[key][diff] = I[key][diff][0] / I[key][diff][1]
        sum_val += p[key][diff]
    for diff in p[key].keys():
        p[key][diff] /= sum_val

In [ ]:
for index in range(train_size):
    length_en = len(tokenized_stores['en_train'][index])
    length_de = len(tokenized_stores['de_train'][index])
    if length_de - length_en > 10 and length_en == 1:
        print("Length of English Sentence:", str(length_en))
        print("Length of German Sentence:", str(length_de))

In [ ]:

init = {}
for length in p:
    max_prob = -1
    max_jump = 0
    for key in p[length].keys():
        if p[length][key] > max_prob:
            max_prob = p[length][key]
            max_jump = key
    init[length] = max_jump

In [ ]:
!pip install nltk

In [ ]:

bigrams = {}
unigrams = {}


def model(dataset_size, dataset_name):
    global bigrams
    global unigrams
    for index in range(dataset_size):
        token_A = ''
        for de_token in tokenized_stores[dataset_name][index]:
            if de_token not in unigrams:
                unigrams[de_token] = 1
            else:
                unigrams[de_token] += 1
            
            token_B = de_token
            if (token_A, token_B) not in bigrams:
                bigrams[(token_A, token_B)] = 1
            else:
                bigrams[(token_A, token_B)] += 1
            token_A = token_B

model(train_size, 'de_train')
model(dev_size, 'de_dev')

bigram_count = len(bigrams)
unigram_count = len(unigrams)
print("Number of Unique Bigrams:", bigram_count)
print("Number of Unique Unigrams:", unigram_count)

Number of Unique Bigrams: 102535
Number of Unique Unigrams: 25429


In [ ]:
from itertools import permutations
import nltk

computed_sentences = []
total_BLEU = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 7: 0}
null_BLEU_count = 0

sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def find_translation(en_token):
    for element in sorted_t:
        if element[0][0].lower() == en_token:
            return element[0][1]
    return ""

def get_prob(seq):
    if len(seq) < 2:
        return 1
    score = 0
    token_A = ''
    for de_token in seq:
        token_B = de_token
        if (token_A, token_B) not in bigrams:
            if token_B not in unigrams:
                continue
            else:
                score += unigrams[token_B] / unigram_count
        else:
            base_token_count = 0
            if token_A in unigrams:
                base_token_count = unigrams[token_A]
            score += (bigrams[(token_A, token_B)] + 1) / (base_token_count + unigram_count)
        token_A = token_B
    return score

count = 0
for index in range(test_size):
    if len(tokenized_stores['en_test'][index]) > 8 or len(tokenized_stores['en_test'][index]) < 2:
        continue

    translated_words = []
    for en_token in tokenized_stores['en_test'][index]:
        translation = find_translation(en_token)
        if translation != "":
            translated_words.append(translation)

    perm = permutations(translated_words)

    best_seq = translated_words
    best_prob = -1

    for seq in perm:
        prob = get_prob(seq)
        if prob > best_prob:
            best_prob = prob
            best_seq = seq

    BLEU_scores = []
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['de_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['de_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method2))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['de_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method3))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['de_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['de_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method5))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['de_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method7))

    for key in total_BLEU.keys():
        if key == 7:
            consider = 5
        else: consider = key - 1
        total_BLEU[key] += BLEU_scores[consider]
    
    if BLEU_scores[0] == 0:
        null_BLEU_count += 1
    
    count += 1
    print("Sentence Index: ", str(count))
    print("English Sentence:", str(tokenized_stores['en_test'][index]))
    print("Reference Hindi Sentence:", str(tokenized_stores['de_test'][index]))
    print("Translated Sentence:", str(best_seq))
    print("Translation BLEU Scores", str(BLEU_scores))
    print()
    
    computed_sentences.append([tokenized_stores['en_test'][index], tokenized_stores['de_test'][index], best_seq, BLEU_scores])

tested = count

Sentence Index:  1
English Sentence: ['People', 'are', 'fixing', 'the', 'roof', 'of', 'a', 'house.']
Reference Hindi Sentence: ['Leute', 'Reparieren', 'das', 'Dach', 'eines', 'Hauses.']
Translated Sentence: ('sind', 'repariert', 'von', 'Haus.', 'den', 'einem', 'über')
Translation BLEU Scores [0, 0, 0, 0, 0, 0]

Sentence Index:  2
English Sentence: ['A', 'guy', 'works', 'on', 'a', 'building.']
Reference Hindi Sentence: ['Ein', 'Typ', 'arbeitet', 'an', 'einem', 'Gebäude.']
Translated Sentence: ('Gebäude.', 'auf', 'arbeitet', 'einem', 'Typ')
Translation BLEU Scores [0.06221008431290531, 0.2810657841493931, 0.1236846477204597, 0.23017347877213462, 0.13655616597520384, 0.2978747055752058]

Sentence Index:  3
English Sentence: ['Three', 'people', 'sit', 'in', 'a', 'cave.']
Reference Hindi Sentence: ['Drei', 'Leute', 'sitzen', 'in', 'einer', 'Höhle.']
Translated Sentence: ('sitzen', 'einem', 'Personen', 'in', 'zu')
Translation BLEU Scores [0.05231223689135342, 0.24736929544091937, 0.104005976

In [ ]:
# Results:
import statistics
print("Number of Samples Tested Upon: " + str(tested))
print()

print("Average BLEU Score using Various Smoothing Functions (considering all test samples)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / tested))
print()
print("Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / (tested - null_BLEU_count)))

Number of Samples Tested Upon: 50 

Average BLEU Score using Various Smoothing Functions (considering all test samples)
Method 1: 0.03836090870318489
Method 2: 0.18765850259273342
Method 3: 0.07607630407313906
Method 4: 0.18308437727795884
Method 5: 0.08731426735847905
Method 7: 0.2373239648107167

Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)
Method 1: 0.04278716739970622
Method 2: 0.20931140673804882
Method 3: 0.08485433915850125
Method 4: 0.20420949773310795
Method 5: 0.09738899051522663
Method 7: 0.26470749921195325
